In [1]:
# DATA_PATH = Path('/data/safegraph/safegraph_open_census_data')
mobility_data_path = '/data/safegraph/social-distancing/2020/06/01/2020-06-01-social-distancing.csv'

In [2]:
time_series_case_counts_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

In [3]:
# https://github.com/obuchel/classification/blob/master/prepare_classification_counties_final.py
# See:  https://obuchel.github.io/classification/output/classification/classification_ids_counties2.json
# https://obuchel.github.io/classification/output/classification/classification_ids_counties2.json
# https://obuchel.github.io/classification/classification_map.html